In [1]:
# 課題　【CSVの読み書き基礎】

# [1] 目的
# 固定の名前リストを CSV に保存し、その CSV を読み返して表示する。
# ファイル入出力の基本（with 構文）と csv モジュールの使い方、
# 日本語環境でつまずきがちな newline / encoding のポイントを実演。

# ------------------------------------------------------------

# [2] （Colab専用）Google Drive をマウント
# ”マウント”とは？
# 👉 Colab（サーバー）の仮想マシンと自分のGoogle Drive をつなげて使える ようになる。
# これで左の「ファイル」タブから /content/drive/MyDrive/ 以下が見えるようになります。

from google.colab import drive
drive.mount('/content/drive')

# これを実行すると：
# ①認証用のURLが表示される
# ②Googleアカウントでログインして認証コードをコピー
# ③Colab側にペースト → /content/drive/ 以下に Drive が接続される
# すでにマウント済みなら "already mounted" と出るだけでOK

# ------------------------------------------------------------

# [3] 保存先フォルダの準備
# Drive上に学習用フォルダを作っておくと管理がラク！

from pathlib import Path

BASE_DIR = Path('/content/drive/MyDrive') / 'Python学習ログ' / 'csv_basics'
BASE_DIR.mkdir(parents=True, exist_ok=True)
CSV_PATH = BASE_DIR / 'names.csv'

print(f'保存先: {CSV_PATH}')

# 例）/content/drive/MyDrive/Python学習ログ/csv_basics/names.csv
# pathlibはPython標準ライブラリのモジュールで、パス操作を直感的に扱えるツール。
# Pathクラスを使うと、文字列の結合を + ではなく / 演算子で表現できるようになる。

# BASE_DIR や CSV_PATH などのようにわかりやすい変数名をつける
# 👉 あとからフォルダ名を変更しても修正が楽になる

# 1) mkdir() … フォルダを作成するメソッド

# BASE_DIR（= Pathオブジェクト）が指すフォルダを作成。
# 作るのはフォルダだけで、ファイルは作らない。

# 2) parents=True

# 親フォルダが無い場合、足りない親もまとめて作るオプション。
# 例：/content/drive/MyDrive/Python学習ログ/csv_basics を作りたいとき、
#「Python学習ログ」が未作成でも自動で作ってから「csv_basics」を作成する。

# False（デフォルト）だと、親が無いと FileNotFoundError になります。

# 3) exist_ok=True

# 目的のフォルダがすでに存在していてもエラーにしない（= 何もしない）。
# False（デフォルト）だと、既存フォルダに対して FileExistsError が出ます。
#「何度実行しても安全」なのが嬉しいポイント。

# ------------------------------------------------------------

# [4] データ定義（固定の名前リスト）
# まずは 1 列だけ（name）の最小構成で始めます。
names = ['Taro', 'Hanako', 'Ken']

# ------------------------------------------------------------

# [5] CSV に書き込む
# ポイント：
#  - newline='' を必ず指定（指定しないと環境によって空行が混ざることがあります）
#  - encoding='utf-8' を指定（日本語名などがあっても文字化けしにくい）

import csv

with open(CSV_PATH, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)

    # csv.writer は、ファイルオブジェクト f に**CSV形式で書くための“書き込み器”**を作る。

    writer.writerow(['name'])

    # ヘッダー（列名）を書いておくと、後で見返したときに意味が分かりやすい
    # 1行分を書き込みます。引数は**シーケンス（リスト/タプル）**で渡すのがルール。

    # writer.writerow(['name']) → 1列1セルの行になります。
    # writer.writerow('name') とすると n, a, m, e に分解されてしまうので注意！
    # この例はヘッダー行（列名）を1行だけ先に書いています。

    # 1 行ずつ書き込み（writerows でまとめて書いてもOK）

    # もし複数列なら？
    # 👉 writer.writerow(['name', 'age'])　とすると、
    # ファイルの1行目は　name,age　となり、2列分のヘッダーに。

    for name in names:
        writer.writerow([name])  # 1列のCSVなので [name] のようにリストで渡す

    # 取り出した name を 1行のリスト にしてCSVに書き込む

    # 注意!：writerow の引数はリスト（[]）で渡す必要がある！

    # writer.writerow([name]) → 正しい（例：["Taro"]）

    # writer.writerow(name) → 間違い（文字列を分解してしまい T, a, r, o になる）

print('✅ CSV に書き込み完了')

# ------------------------------------------------------------

# [6] CSV を読み返す
# csv.reader は各行を「リスト」で返します（すべて文字列型です）。

loaded = []
with open(CSV_PATH, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)          # 先頭行をヘッダーとして読み飛ばす（['name']）
    for row in reader:
        if not row:                # 空行対策（ないはずだが念のため）
            continue
        loaded.append(row[0])      # 1列目（name）だけを取り出して保存

# loaded = [] 👉 読み込んだ名前を入れていく「空のリスト」
# header = next(reader)
# 先頭行（ここでは ['name']）を読み取って、変数 header に入れています。
# next(reader) は「イテレータから1つ取り出す」という命令。
# こうしておくと、後続のループは 2行目以降から始まります。


print('✅ CSV から読み込んだ結果：')
for i, name in enumerate(loaded, start=1):
    print(f'{i}. {name}')

# enumerate は「リストの要素を取り出すと同時に、その番号（インデックス）も返してくれる関数」。
# loaded は、さっき読み込んだ名前のリスト（例: ["Taro", "Hanako", "Ken"]）。

# 通常の for name in loaded: だと name だけしか取り出せませんが、
# enumerate(loaded, start=1) を使うと：

# 1回目 → i=1, name="Taro"
# 2回目 → i=2, name="Hanako"
# 3回目 → i=3, name="Ken"

# 👉 start=1 は「番号を 1 から始めたい」指定です。
# デフォルト（省略時）は 0 から始まります。

# print(f'{i}. {name}')
# f文字列（フォーマット済み文字列リテラル）。
# {i} と {name} が、それぞれ変数 i と name の値に展開されます。

# ------------------------------------------------------------

# [7] 検証（往復でデータが一致するか）
# 失敗すると AssertionError を出して気づけます（テストの最小形）。
assert loaded == names, '読み返した名前リストが元データと一致しません'
print('✅ 検証OK: 書き込み→読み込みの往復でデータ一致')

# assert 条件式, 'エラーメッセージ'
# assert は「もし条件が False だったらエラーを出す」というチェック用の命令です。

# ここでは：
# assert loaded == names, '読み返した名前リストが元データと一致しません'
# → 「loaded（読み込んだリスト）と names（元のリスト）が等しいか」を確認しています。

# 条件が True（等しい） → 何も起きずに次の行へ進む。
# 条件が False（違う） → AssertionError が発生し、指定したエラーメッセージが表示される。

# 2) print('✅ 検証OK: 書き込み→読み込みの往復でデータ一致')

# assert が成功した（エラーにならなかった）場合だけ実行されます。
#「CSVに書いたものと、読み返したものが完全一致しましたよ！」と確認メッセージを出しています。

# ------------------------------------------------------------

# [8] ここまでの要点まとめ（メモ）
# - with open(...) を使うと、処理の最後に自動でファイルが閉じられます（閉じ忘れ防止）。
# - newline='' は CSV でほぼ必須。未指定だと空行が挟まる事例があります。
# - CSV から読み込んだ値は str（文字列）になります。数値は int(...) などで変換します。
# - 左の「ファイル」タブを更新（↻）→ Drive → MyDrive → Python学習ログ → csv_basics から names.csv を確認できます。


Mounted at /content/drive
保存先: /content/drive/MyDrive/Python学習ログ/csv_basics/names.csv
✅ CSV に書き込み完了
✅ CSV から読み込んだ結果：
1. Taro
2. Hanako
3. Ken
✅ 検証OK: 書き込み→読み込みの往復でデータ一致
